In [12]:
import psycopg2
from psycopg2 import sql

# Функция для подключения к базе данных
def connect_to_db():
    try:
        conn = psycopg2.connect(
            dbname="Airport_DB",  # Название  базы данных
            user="postgres",  # имя пользователя
            password="1234509876",  # пароль
            host="localhost",  # Хост
            port="5432"  # Порт
        )
        return conn
    except Exception as e:
        print(f"Ошибка подключения к базе данных: {e}")
        return None


In [13]:

# Функция для просмотра доступных рейсов
def show_available_flights(conn):
    try:
        cur = conn.cursor()
        query = sql.SQL("""
            SELECT * FROM Flights WHERE status = 'Scheduled'
        """)
        cur.execute(query)
        flights = cur.fetchall()
        if flights:
            print("\nДоступные рейсы:")
            for flight in flights:
                print(f"ID: {flight[0]}, Номер рейса: {flight[1]}, Вылет: {flight[2]}, Прилет: {flight[3]}, Самолет: {flight[4]}, Гейт: {flight[5]}")
        else:
            print("Нет доступных рейсов.")
    except Exception as e:
        print(f"Ошибка при получении рейсов: {e}")


In [14]:

# Функция для регистрации пассажира
def register_passenger(conn, first_name, last_name, passport_number):
    try:
        cur = conn.cursor()
        query = sql.SQL("""
            INSERT INTO Passengers (first_name, last_name, passport_number)
            VALUES (%s, %s, %s)
            RETURNING passenger_id
        """)
        cur.execute(query, (first_name, last_name, passport_number))
        passenger_id = cur.fetchone()[0]
        conn.commit()
        print("Пассажир успешно зарегистрирован!")
        return passenger_id
    except Exception as e:
        print(f"Ошибка при регистрации пассажира: {e}")
        return None


In [15]:

# Функция для покупки билета
def buy_ticket(conn, flight_id, passenger_id, seat_number, ticket_class):
    try:
        cur = conn.cursor()
        query = sql.SQL("""
            INSERT INTO Tickets (flight_id, passenger_id, seat_number, class)
            VALUES (%s, %s, %s, %s)
        """)
        cur.execute(query, (flight_id, passenger_id, seat_number, ticket_class))
        conn.commit()
        print("Билет успешно куплен!")
    except Exception as e:
        print(f"Ошибка при покупке билета: {e}")


In [16]:

# Основное меню программы
def main_menu(conn):
    while True:
        print("\nМеню:")
        print("1. Показать доступные рейсы")
        print("2. Купить билет")
        print("3. Выйти")
        choice = input("Выберите действие: ")

        if choice == "1":
            show_available_flights(conn)

        elif choice == "2":
            # Покупка билета
            flight_id = input("Введите ID рейса: ")
            first_name = input("Введите ваше имя: ")
            last_name = input("Введите вашу фамилию: ")
            passport_number = input("Введите номер паспорта: ")
            seat_number = input("Введите номер места: ")
            ticket_class = input("Введите класс обслуживания (Economy/Business/First): ")

            # Проверяем, зарегистрирован ли пассажир
            cur = conn.cursor()
            cur.execute("SELECT passenger_id FROM Passengers WHERE passport_number = %s", (passport_number,))
            passenger = cur.fetchone()

            if passenger:
                passenger_id = passenger[0]
                print("Пассажир уже зарегистрирован.")
            else:
                passenger_id = register_passenger(conn, first_name, last_name, passport_number)

            if passenger_id:
                buy_ticket(conn, flight_id, passenger_id, seat_number, ticket_class)

        elif choice == "3":
            print("Выход из программы.")
            break

        else:
            print("Неверный выбор. Попробуйте снова.")


In [17]:

# Основная функция
def main():
    conn = connect_to_db()
    if conn:
        main_menu(conn)
        conn.close()
    else:
        print("Не удалось подключиться к базе данных.")


In [18]:

# Запуск программы
if __name__ == "__main__":
    main()


Меню:
1. Показать доступные рейсы
2. Купить билет
3. Выйти


Выберите действие:  1



Доступные рейсы:
ID: 10, Номер рейса: FL138, Вылет: 2025-07-14 22:35:10.578367, Прилет: 2026-01-04 06:40:46.612882, Самолет: 16, Гейт: 34
ID: 16, Номер рейса: FL914, Вылет: 2025-04-27 17:45:48.336733, Прилет: 2025-11-24 15:51:59.764856, Самолет: 44, Гейт: 19
ID: 22, Номер рейса: FL239, Вылет: 2025-08-11 04:50:43.857090, Прилет: 2025-08-24 11:44:31.537054, Самолет: 20, Гейт: 46
ID: 29, Номер рейса: FL378, Вылет: 2025-07-26 20:54:37.786629, Прилет: 2025-04-18 23:35:33.497912, Самолет: 2, Гейт: 35
ID: 30, Номер рейса: FL631, Вылет: 2025-09-25 21:37:30.042262, Прилет: 2025-03-11 05:13:14.127498, Самолет: 15, Гейт: 40
ID: 33, Номер рейса: FL783, Вылет: 2025-02-27 15:41:26.723275, Прилет: 2025-10-21 06:24:01.439674, Самолет: 5, Гейт: 42
ID: 38, Номер рейса: FL120, Вылет: 2025-02-20 00:59:40.195223, Прилет: 2025-07-04 20:19:45.448080, Самолет: 37, Гейт: 37
ID: 40, Номер рейса: FL876, Вылет: 2025-06-21 00:35:35.330589, Прилет: 2025-04-23 17:50:38.684296, Самолет: 18, Гейт: 20
ID: 41, Номер ре

Выберите действие:  2
Введите ID рейса:  10
Введите ваше имя:  Амантур
Введите вашу фамилию:  Самаганов
Введите номер паспорта:  123456789012345
Введите номер места:  US
Введите класс обслуживания (Economy/Business/First):  Economy


Пассажир успешно зарегистрирован!

Меню:
1. Показать доступные рейсы
2. Купить билет
3. Выйти


Выберите действие:  3


Выход из программы.
